## 张量

### 3.4 张量命名

In [ ]:
import torch
img_t = torch.randn(3, 5, 5)  # shape [channels, rows, columns]
weights = torch.tensor([0.2126, 0.7152, 0.0722]) # shape [channels]

我们通常希望我们的代码具有通用性（例如引入三个channels以保证彩色图像的处理）；同样，为了确保处理多张图片，所以引入batch的概念

In [3]:
# 为什么要引入batch？ 因为我们通常不会只有一个图片，而是一批图片
batch_t = torch.randn(2, 3, 5, 5)  # shape [batch, channels, rows, columns]

In [ ]:
img_gray_naive = img_t.mean(-3)
batch_gray_naive = batch_t.mean(-3) # 这里的-3是什么意思？ 为什么要用-3？ 为什么要用mean？
# -3 表示在第三个维度上进行操作，也就是channels这个维度上 
# mean 表示求平均值，也就是把所有channels的值加起来，然后除以channels的数量
# 为什么要用mean？ 因为我们要把所有channels的值加起来，然后除以channels的数量，这样就得到了灰度图
print(img_gray_naive.shape, batch_gray_naive.shape)

torch.Size([5, 5]) torch.Size([2, 5, 5])


但是这方法会造成混乱：
- 我们不能确定到底哪个是channels
- 写代码的时候会乱掉，通用性差

In [10]:
# 为了解决这一问题，Pytorch引入了einsum
img_gray = torch.einsum('...chw,c->...hw', img_t, weights)
batch_gray = torch.einsum('...chw,c->...hw', batch_t, weights)
print(img_gray.shape, batch_gray.shape)

torch.Size([5, 5]) torch.Size([2, 5, 5])


但是这个sb方法很少用，不用管他

In [13]:
# 给维度引入命名规则？？是不是一个解决方案？
weights_named = torch.tensor([0.2126, 0.7152, 0.0722], names=['channels']) # name是一个维度的名字，这样我们就可以通过名字来引用维度，而不是通过维度的位置
weights_named

tensor([0.2126, 0.7152, 0.0722], names=('channels',))

In [24]:
img_named = img_t.refine_names(..., 'channels', 'rows', 'columns')
batch_named = batch_t.refine_names(..., 'channels', 'rows', 'columns')
print("img named:", img_named.shape, img_named.names)
print("batch named:", batch_named.shape, batch_named.names)

img named: torch.Size([3, 5, 5]) ('channels', 'rows', 'columns')
batch named: torch.Size([2, 3, 5, 5]) (None, 'channels', 'rows', 'columns')


In [25]:
weights_aligned = weights_named.align_as(img_named) # align_as是什么意思？
print(weights_aligned.shape, weights_aligned.names)

torch.Size([3, 1, 1]) ('channels', 'rows', 'columns')


### 3.5 张量的元素类型

dytype：张量中的数据类型，有以下几种：
- torch.float32: 单精度浮点（最常使用）
- torch.float64: 双精度浮点
- torch.16 or torch.half: 半精度浮点
- torch.int8: 8位有符号整数
- torch.unit8: 8位无符号整数
- torch.int16 ot torch.short: 16位有符号整数
- torch.int32 ot torch.int: 32位有符号整数
- torch.int64 ot torch.long: 64位有符号整数
- torch.bool: 布尔形

调整单精度/双精度/半精度对神经网络模型影响很小

我们可以在创建张量的时候直接定义dtype

In [28]:
double_points = torch.ones(10, 2, dtype=torch.double)
short_points = torch.tensor([[1, 2], [3, 4]], dtype=torch.short)
print(double_points.dtype, short_points.dtype)

torch.float64 torch.int16


我们也可以用函数转化为我们想要的数据类型

In [29]:
double_points = torch.zeros(10, 2).double()
short_points = torch.ones(10, 2).short()
print(double_points.dtype, short_points.dtype)

torch.float64 torch.int16


此外，更通用的，我们可以使用to函数

In [30]:
double_points = torch.zeros(10, 2).to(torch.double)
short_points = torch.ones(10, 2).to(torch.short)
print(double_points.dtype, short_points.dtype)

torch.float64 torch.int16


### 3.6 张量的API

In [31]:
a = torch.ones(3, 2)
a_t = torch.transpose(a, 0, 1)
print(a.shape, a_t.shape)

torch.Size([3, 2]) torch.Size([2, 3])


也可以作为张量的一个method使用：

In [32]:
a = torch.ones(3, 2)
a_t = a.transpose(0, 1)
print(a.shape, a_t.shape)

torch.Size([3, 2]) torch.Size([2, 3])


In [34]:
b = torch.ones(3, 2, 5)
b_t = b.transpose(0, 2)
print(b.shape, b_t.shape)

torch.Size([3, 2, 5]) torch.Size([5, 2, 3])


### 3.7 张量的储存试图